In [ ]:
import os, sys, pandas as pd, json, copy, numpy as np
sys.path.append("/content")
from src.utils import plot_obs_predict
from sklearn.metrics import r2_score
os.chdir("/content")
with open("/content/credentials") as f:
    env_vars = f.read().split("\n")

for var in env_vars:
    key, value = var.split(" = ")
    os.environ[key] = value

storage_options = {"account_name":os.environ["ACCOUNT_NAME"],
                   "account_key":os.environ["BLOB_KEY"]}

In [ ]:
results = pd.read_csv("output/mlp/grid_search_metadata_v3.csv")
no_azimuth = ["mean_viewing_azimuth" not in results["features"][i] for i in range(0, (results.shape[0]))]
results_sub = copy.copy(results.loc[no_azimuth, :])

results_to_use = results_sub
results_to_use["mean_mse"] = (results_to_use["val_site_mse"] + results_to_use["val_pooled_mse"]) / 2

with open(results_to_use.iloc[results_to_use["val_pooled_mse"].argmin(), :]["path"], "rb") as f:
    top_model = json.load(f)

In [ ]:
part = "train"
scale = "native"
my_fold = 1

from matplotlib import pyplot as plt

if part == "validation":
    pred_fold = top_model["y_pred_val_fold"][0]
    obs_fold = top_model["y_obs_val_fold"][0]
else:
    pred_fold = top_model["y_pred_train_fold"][0]
    obs_fold = top_model["y_obs_train_fold"][0]


if scale == "log":
    pred = pred_fold
    obs = obs_fold
    units = "log(SSC)"
else:
    pred = [np.exp(x) for x in pred_fold]
    obs = [np.exp(x) for x in obs_fold]
    units = "SSC"

print(f"R2: {r2_score(pred, obs)}")
obs_pred = pd.DataFrame({"pred": pred, "obs":obs})
plot_obs_predict(obs_pred, f"Observed vs. Predicted, Fold {my_fold+1}", units)
